In [14]:
import json
import pandas as pd
df1 = pd.read_csv("post_data.csv")
df2 = pd.read_csv("replies_data.csv")

with open("output.json", "rb") as f:
    json_iniziale = json.load(f)

In [15]:
import pandas as pd

def aggiorna_json(json_iniziale, df1, df2,colonne_selezionate = ['Post ID','User ID','Thread Type','Username','Like Count','Caption Text']):
    """
    Aggiorna un JSON con dati estratti da due dataframe basandosi sugli ID presenti nel JSON.

    :param json_iniziale: Dict contenente i dati iniziali
    :param df1: DataFrame con colonne ['Post ID', 'User ID', 'Thread Type', 'Username', 'Like Count', 'Caption Text']
    :param df2: DataFrame con colonne ['Post ID', 'User ID', 'Thread Type', 'Username', 'Like Count', 'Caption Text']
    :return: JSON aggiornato
    """
    df1 = df1[colonne_selezionate]
    df2 = df2[colonne_selezionate]   
    
    # Unione dei dataframe
    df_totale = pd.concat([df1, df2])

    # Funzione per trovare dati di un utente in base all'ID
    def estrai_dati(user_id):
        dati = df_totale[df_totale['User ID'] == int(user_id)]
        if not dati.empty:
            return dati.to_dict(orient='records')  # Ritorna una lista di dizionari
        return []  # Se non ci sono dati

    # Itera sugli ID principali nel JSON
    for user_id, user_data in json_iniziale.items():
        # Aggiungi i dati dell'utente esterno
        json_iniziale[user_id]['user_data'] = estrai_dati(user_id)

        # Itera sui follower
        for follower in user_data.get('followers', []):
            follower_id = follower['user_id']
            follower['user_data'] = estrai_dati(follower_id)

    return json_iniziale


In [16]:
# Chiamata al metodo
json_aggiornato = aggiorna_json(json_iniziale, df1, df2)

In [17]:
with open("final_dataset.json", "w", encoding="utf-8") as f2:
    json.dump(json_aggiornato, f2, ensure_ascii=False, indent=4)